# Recognizing Flatlanders

## Data preprocessing

In [ ]:
import h5py
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

Read the training and test data from the HDF5 files.

In [ ]:
with h5py.File('train_data.h5', 'r') as h5_file:
    x_train = np.array(h5_file['x_values'])
    y_train = np.array(h5_file['y_values'])
with h5py.File('test_data.h5', 'r') as h5_file:
    x_test = np.array(h5_file['x_values'])
    y_test = np.array(h5_file['y_values'])

Determine whether the backend expects the channel to come first or last in the input shape.

In [ ]:
img_channels, img_rows, img_cols = 1, x_train.shape[1], x_train.shape[2]

if K.image_data_format() == 'channels_first':
    shape_ord = (img_channels, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, img_channels)

Reshape the data accordingly.

In [ ]:
x_train = x_train.reshape((x_train.shape[0], ) + shape_ord)
x_test = x_test.reshape((x_test.shape[0], ) + shape_ord)

Convert the input data to single precision floating point, and scale between 0 and 1.

In [ ]:
x_train = x_train.astype(np.float32)/255.0
x_test = x_test.astype(np.float32)/255.0

Verify input data format and shape.

In [ ]:
print(x_train.shape, x_train.dtype, x_train.min(), x_train.max())
print(x_test.shape, x_test.dtype, x_test.min(), x_test.max())

Verify output data format and shape.

In [ ]:
print(y_train.shape, y_train.dtype, y_train.min(), y_train.max())
print(y_test.shape, y_test.dtype, y_test.min(), y_test.max())

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

In [ ]:
print(x_train.shape, x_train.dtype, x_train.min(), x_train.max())
print(x_val.shape, x_val.dtype, x_val.min(), x_val.max())

In [ ]:
print(y_train.shape, y_train.dtype, y_train.min(), y_train.max())
print(y_val.shape, y_val.dtype, y_val.min(), y_val.max())

## Model definition

In [ ]:
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, LeakyReLU
from keras.layers.pooling import MaxPool2D
from keras.models import Sequential
from keras.optimizers import SGD, Adam

In [ ]:
nr_filters = 32
nr_classes = y_train.shape[-1]
conv_x_size, conv_y_size = 5, 5
model = Sequential()
model.add(Conv2D(nr_filters, (conv_x_size, conv_y_size), strides=2,
                 padding='valid', input_shape=shape_ord))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Conv2D(nr_filters*2, (conv_x_size, conv_y_size), strides=2,
                 padding='valid'))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Conv2D(nr_filters*4, (conv_x_size, conv_y_size), strides=2,
                 padding='valid'))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(nr_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=20, batch_size=64,
                    validation_data=(x_val, y_val))

In [ ]:
model.evaluate(x_test, y_test)